In [281]:
!pip install --quiet --upgrade langchain langchain-community langchain-chroma

In [307]:
!pip install -qU langchain-openai
os.environ["LANGCHAIN_TRACING_V2"] = "false"  
os.environ["OPENAI_API_KEY"] = getpass.getpass()
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

 ········


In [309]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

In [311]:
!pip install pypdf
import PyPDF2

pdf_loader = PyPDFLoader(file_path='Resilient_LosAngeles_Plan.pdf') 
docs = pdf_loader.load()

In [312]:
len(docs[0].page_content)

91

In [313]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
len(all_splits)

468

In [314]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [327]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What Climate Threats?")

len(retrieved_docs)

6

In [329]:
print(retrieved_docs[0].page_content)

Recent catastrophes around the world —
hurricanes in the U.S. and the Caribbean, 
earthquakes in Mexico, fires in the western U.S., 
flooding in Nepal and Bangladesh —are serious 
reminders that every community needs focused resilience strategies. 
In Los Angeles, we know a major earthquake is 
a matter of when, not if. And we know that the 
next one could be more than 10 times worse 
than the 1994 Northridge earthquake that 
claimed 57 lives, injured over 9,000, displaced 
tens of thousands from their homes, and caused 
$20 billion in damage. 
And earthquakes aren’t the only threat we face: 
The effects of climate change are worsening, 
and cybercrime is causing more turmoil than 
ever for businesses, governments, and other 
institutions around the world. These threats are emerging quickly, and 
we can’t wait for catastrophes to hit before 
confronting them. We need a comprehensive, 
strategically coordinated approach to  
urban resilience.


In [317]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [318]:
definitions = {
    "Climate Change": "A long-term shift in weather patterns and temperatures, primarily caused by human activities emitting greenhouse gases (GHGs).",
    "Greenhouse Gases (GHGs)": "Atmospheric gases like CO₂, CH₄, N₂O that absorb and emit radiation, leading to the greenhouse effect.",
    "Anthropogenic Emissions": "Emissions of GHGs resulting from human activities such as burning fossil fuels and deforestation.",
    "Climate Impacts": "Consequences of climate-related hazards on natural and human systems, affecting lives, ecosystems, economies, and infrastructure.",
    "Climate Risk": "Potential negative consequences from climate impacts, resulting from the interaction of hazard, exposure, and vulnerability.",
    "Climate Vulnerability": "The degree to which a system is susceptible to harm from climate change and its ability to adapt.",
    "Climate Policies": "Strategies and measures adopted to implement resilience, mitigation, and adaptation options.",
    "Resilience": "The ability of systems to cope with climate hazards by maintaining essential functions and adapting to changes.",
    "Resilience Options": "Strategies to build resilience through policy changes, infrastructure improvements, planning, etc.",
    "Mitigation": "Efforts to reduce or prevent emission of GHGs.",
    "Mitigation Options": "Technologies or practices that contribute to mitigation, like renewable energy or waste minimization.",
    "Adaptation": "Adjusting systems to actual or expected climate changes to minimize harm or exploit beneficial opportunities.",
    "Adaptation Options": "Strategies addressing climate change adaptation, including structural, institutional, ecological, and behavioral measures.",
    "Climate Justice": "Ensuring equitable sharing of the burdens and benefits of climate change impacts.",
    "Maladaptation": "Actions that may increase vulnerability to climate change or diminish resilience.",
    "Scenario": "A plausible description of how the future may develop based on a coherent set of assumptions."
}

In [322]:
normalized_definitions = {key.lower(): value for key, value in definitions.items()}
definitions_context = "\n".join([f"{key}: {value}" for key, value in definitions.items()])


In [345]:
system_prompt = (
    "You are a researcher specializing in extracting climate-related information from climate adaptation and resilience plans."
    "You are provided with a document (referred to as the 'plan') from a state or city in the United States (the 'planning jurisdiction')that has adopted policies addressing climate change, which may pertain to resilience, mitigation, and adaptation."
    "\n\n"
    "{context}"
    "Definitions:\n"
    f"{definitions_context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
print("Prompt Structure:", prompt)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


Prompt Structure: input_variables=['context', 'input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are a researcher specializing in extracting climate-related information from climate adaptation and resilience plans.You are provided with a document (referred to as the 'plan') from a state or city in the United States (the 'planning jurisdiction')that has adopted policies addressing climate change, which may pertain to resilience, mitigation, and adaptation.\n\n{context}Definitions:\nClimate Change: A long-term shift in weather patterns and temperatures, primarily caused by human activities emitting greenhouse gases (GHGs).\nGreenhouse Gases (GHGs): Atmospheric gases like CO₂, CH₄, N₂O that absorb and emit radiation, leading to the greenhouse effect.\nAnthropogenic Emissions: Emissions of GHGs resulting from human activities such as burning fossil fuels an

In [324]:
# List of questions to query

context_questions = [
     #Planning Jurisdiction
    "Provide a summary of information available from the plan regarding the planning jurisdiction, including its name, location, population, geography, major industries, per capita income, and similar information.",
    #Plan Scope
    "To what extent does the plan address resilience, mitigation, and adaptation?",
    #Scenario
    "Does the plan describe a scenario of climate change the planning jurisdiction will face and for which it is planning climate policies for resilience, mitigation, and adaptation? If so, please summarize any such scenario."
]

threats_questions = [
    #Climate Impacts
    """What climate impacts does the plan identify as presenting climate risks for the planning jurisdiction? For each identified climate impact, summarize the plan’s description of:
    1. The climate impact.
    2. The climate risks it poses to the planning jurisdiction.
    3. Areas or populations with the most climate vulnerability to the identified climate impact.""",
]

measures_questions = [
    #Resilience Options
    "What resilience options does the plan identify? For each identified resilience option, summarize the information provided in the plan regarding cost, timing, location, and other important attributes.",
    #Mitigation Options
    "What mitigation options does the plan identify? For each identified mitigation option, summarize the information provided in the plan regarding cost, timing, location, and other important attributes."
]

questions = [
    #Planning Jurisdiction
    "Provide a summary of information available from the plan regarding the planning jurisdiction, including its name, location, population, geography, major industries, per capita income, and similar information.",
    #Plan Scope
    "To what extent does the plan address resilience, mitigation, and adaptation?",
    #Scenario
    "Does the plan describe a scenario of climate change the planning jurisdiction will face and for which it is planning climate policies for resilience, mitigation, and adaptation? If so, please summarize any such scenario.",
    #Climate Impacts
    """What climate impacts does the plan identify as presenting climate risks for the planning jurisdiction? For each identified climate impact, summarize the plan’s description of:
    1. The climate impact.
    2. The climate risks it poses to the planning jurisdiction.
    3. Areas or populations with the most climate vulnerability to the identified climate impact.""",
    #Resilience Options
    "What resilience options does the plan identify? For each identified resilience option, summarize the information provided in the plan regarding cost, timing, location, and other important attributes.",
    #Mitigation Options
    "What mitigation options does the plan identify? For each identified mitigation option, summarize the information provided in the plan regarding cost, timing, location, and other important attributes."
]

In [349]:
def generate_responses(questions):
    responses = []
    for question in questions:
        try:
            response = rag_chain.invoke({"input": question.lower()})
            responses.append(f"**{question}**\n{response['answer']}\n\n")
        except Exception as e:
            responses.append(f"**{question}**\nError: {str(e)}\n\n")
    return responses

In [ ]:
Output = "# Climate Change Responses\n\n"

Output += "# Context Questions\n"
Output += (generate_responses(context_questions))

print(Output)

In [367]:
output_context_questions = "# Climate Change Responses\n\n"

output_context_questions += "# Context Questions\n"
output_context_questions += str(generate_responses(context_questions))

print(output_context_questions)

Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Forbidden"}')
Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Forbidden"}')
Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Forbidden"}')
Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{

# Climate Change Responses

# Context Questions
["**Provide a summary of information available from the plan regarding the planning jurisdiction, including its name, location, population, geography, major industries, per capita income, and similar information.**\nThe provided text does not contain specific details regarding the planning jurisdiction's name, location, population, geography, major industries, or per capita income. It primarily focuses on the incorporation of climate adaptation, hazard mitigation, and resilience strategies within the General Plan of the jurisdiction, as well as the involvement of various partners in this process.\n\nTo gather the requested summary about the planning jurisdiction, additional information from the complete climate adaptation and resilience plan or other relevant documents would be necessary. If you have more content from the plan or other resources, please provide it so I can assist you further.\n\n", '**To what extent does the plan address 

In [351]:
markdown_output += "# Threats Questions\n"
markdown_output += "".join(generate_responses(threats_questions))

print(markdown_output)

Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Forbidden"}')
Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Forbidden"}')


# Climate Change Responses

# Context Questions
**Provide a summary of information available from the plan regarding the planning jurisdiction, including its name, location, population, geography, major industries, per capita income, and similar information.**
The provided text does not explicitly mention the name of the planning jurisdiction, its location, population, geography, major industries, or per capita income. It primarily focuses on the guiding principles for the General Plan, emphasizing climate adaptation, hazard mitigation, recovery, equity, and infrastructure investment. 

To summarize, the plan outlines:

- **Focus Areas**: Climate adaptation, hazard mitigation, recovery, equity, and long-term capital planning for infrastructure.
- **Objectives**: To enhance citywide resilience by informing land-use decisions that help individuals, neighborhoods, economic systems, and the environment recover from disasters, climate change, and economic shifts.
- **Partnerships**: Involve

In [352]:
markdown_output += "# Measure Questions\n"
markdown_output += "".join(generate_responses(measures_questions))

print(markdown_output)


Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Forbidden"}')
Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Forbidden"}')
Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Forbidden"}')
Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{

# Climate Change Responses

# Context Questions
**Provide a summary of information available from the plan regarding the planning jurisdiction, including its name, location, population, geography, major industries, per capita income, and similar information.**
The provided text does not explicitly mention the name of the planning jurisdiction, its location, population, geography, major industries, or per capita income. It primarily focuses on the guiding principles for the General Plan, emphasizing climate adaptation, hazard mitigation, recovery, equity, and infrastructure investment. 

To summarize, the plan outlines:

- **Focus Areas**: Climate adaptation, hazard mitigation, recovery, equity, and long-term capital planning for infrastructure.
- **Objectives**: To enhance citywide resilience by informing land-use decisions that help individuals, neighborhoods, economic systems, and the environment recover from disasters, climate change, and economic shifts.
- **Partnerships**: Involve